In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import PIL.Image as Image
import cv2
import copy
import time

In [2]:
NUM_CLASS=3
CROP_SIZE=160
NUM_FRAMES_PER_CLIP=16
BATCH_SIZE=10
RGB_CHANNEL=3
IS_TRAIN=True
BLOCK_EXPANSION=4

In [3]:
def get_conv_weight(name,kshape,wd=0.0005*0.3*0.3):
    with tf.device('/cpu:0'):
        var=tf.get_variable(name,shape=kshape,initializer=tf.truncated_normal_initializer(stddev=0.01))
    if wd!=0:
        weight_decay = tf.nn.l2_loss(var)*wd
        tf.add_to_collection('weightdecay_losses', weight_decay)
    return var

In [4]:
def convS(name,l_input,in_channels,out_channels):
    return tf.nn.bias_add(tf.nn.conv3d(l_input,get_conv_weight(name=name,
                                                               kshape=[1,3,3,in_channels,out_channels]),
                                                               strides=[1,1,1,1,1],padding='SAME'),
                                              get_conv_weight(name+'_bias',[out_channels],0))
def convT(name,l_input,in_channels,out_channels):
    return tf.nn.bias_add(tf.nn.conv3d(l_input,get_conv_weight(name=name,
                                                               kshape=[3,1,1,in_channels,out_channels]),
                                                               strides=[1,1,1,1,1],padding='SAME'),
                                              get_conv_weight(name+'_bias',[out_channels],0))

In [5]:
class Bottleneck():
    def __init__(self,l_input,inplanes,planes,stride=1,downsample='',n_s=0,depth_3d=47):
        
        self.X_input=l_input
        self.downsample=downsample
        self.planes=planes
        self.inplanes=inplanes
        self.depth_3d=depth_3d
        self.ST_struc=('A','B','C')
        self.len_ST=len(self.ST_struc)
        self.id=n_s
        self.n_s=n_s
        self.ST=list(self.ST_struc)[self.id % self.len_ST]
        self.stride_p=[1,1,1,1,1]
       
        if self.downsample!='':
            self.stride_p=[1,1,2,2,1]
        if n_s<self.depth_3d:
            if n_s==0:
                self.stride_p=[1,1,1,1,1]
        else:
            if n_s==self.depth_3d:
                self.stride_p=[1,2,2,2,1]
            else:
                self.stride_p=[1,1,1,1,1]
    #P3D has three types of bottleneck sub-structions.
    def ST_A(self,name,x):
        x=convS(name+'_S',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        x=convT(name+'_T',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        return x
    
    def ST_B(self,name,x):
        tmp_x=convS(name+'_S',x,self.planes,self.planes)
        tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
        tmp_x=tf.nn.relu(tmp_x)
        x=convT(name+'_T',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        return x+tmp_x
    
    def ST_C(self,name,x):
        x=convS(name+'_S',x,self.planes,self.planes)
        x=tf.layers.batch_normalization(x,training=IS_TRAIN)
        x=tf.nn.relu(x)
        tmp_x=convT(name+'_T',x,self.planes,self.planes)
        tmp_x=tf.layers.batch_normalization(tmp_x,training=IS_TRAIN)
        tmp_x=tf.nn.relu(tmp_x)
        return x+tmp_x
    
    def infer(self):
        residual=self.X_input
        if self.n_s<self.depth_3d:
            out=tf.nn.conv3d(self.X_input,get_conv_weight('conv3_{}_1'.format(self.id),[1,1,1,self.inplanes,self.planes]),
                             strides=self.stride_p,padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
            
        else:
            param=self.stride_p
            param.pop(1)
            out=tf.nn.conv2d(self.X_input,get_conv_weight('conv2_{}_1'.format(self.id),[1,1,self.inplanes,self.planes]),
                             strides=param,padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
    
        out=tf.nn.relu(out)    
        if self.id<self.depth_3d:
            if self.ST=='A':
                out=self.ST_A('STA_{}_2'.format(self.id),out)
            elif self.ST=='B':
                out=self.ST_B('STB_{}_2'.format(self.id),out)
            elif self.ST=='C':
                out=self.ST_C('STC_{}_2'.format(self.id),out)
        else:
            out=tf.nn.conv2d(out,get_conv_weight('conv2_{}_2'.format(self.id),[3,3,self.planes,self.planes]),
                                  strides=[1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
            out=tf.nn.relu(out)

        if self.n_s<self.depth_3d:
            out=tf.nn.conv3d(out,get_conv_weight('conv3_{}_3'.format(self.id),[1,1,1,self.planes,self.planes*BLOCK_EXPANSION]),
                             strides=[1,1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
        else:
            out=tf.nn.conv2d(out,get_conv_weight('conv2_{}_3'.format(self.id),[1,1,self.planes,self.planes*BLOCK_EXPANSION]),
                             strides=[1,1,1,1],padding='SAME')
            out=tf.layers.batch_normalization(out,training=IS_TRAIN)
           
        if len(self.downsample)==1:
            residual=tf.nn.conv2d(residual,get_conv_weight('dw2d_{}'.format(self.id),[1,1,self.inplanes,self.planes*BLOCK_EXPANSION]),
                                  strides=[1,2,2,1],padding='SAME')
            residual=tf.layers.batch_normalization(residual,training=IS_TRAIN)
        elif len(self.downsample)==2:
            residual=tf.nn.conv3d(residual,get_conv_weight('dw3d_{}'.format(self.id),[1,1,1,self.inplanes,self.planes*BLOCK_EXPANSION]),
                                  strides=self.downsample[1],padding='SAME')
            residual=tf.layers.batch_normalization(residual,training=IS_TRAIN)
        out+=residual
        out=tf.nn.relu(out)
        
        return out

In [6]:
class make_block():
    def __init__(self,_X,planes,num,inplanes,cnt,depth_3d=47,stride=1):
        self.input=_X
        self.planes=planes
        self.inplanes=inplanes
        self.num=num
        self.cnt=cnt
        self.depth_3d=depth_3d
        self.stride=stride
        if self.cnt<depth_3d:
            if self.cnt==0:
                stride_p=[1,1,1,1,1]
            else:
                stride_p=[1,1,2,2,1]
            if stride!=1 or inplanes!=planes*BLOCK_EXPANSION:
                self.downsample=['3d',stride_p]
        else:
            if stride!=1 or inplanes!=planes*BLOCK_EXPANSION:
                self.downsample=['2d']
    def infer(self):
        x=Bottleneck(self.input,self.inplanes,self.planes,self.stride,self.downsample,n_s=self.cnt,depth_3d=self.depth_3d).infer()
        self.cnt+=1
        self.inplanes=BLOCK_EXPANSION*self.planes
        for i in range(1,self.num):
            x=Bottleneck(x,self.inplanes,self.planes,n_s=self.cnt,depth_3d=self.depth_3d).infer()
            self.cnt+=1
        return x

In [7]:
def inference_p3d(_X):
    cnt=0
    conv1_custom=tf.nn.conv3d(_X,get_conv_weight('firstconv1',[1,7,7,RGB_CHANNEL,64]),strides=[1,1,2,2,1],padding='SAME')
    conv1_custom_bn=tf.layers.batch_normalization(conv1_custom,training=IS_TRAIN)
    conv1_custom_bn_relu=tf.nn.relu(conv1_custom_bn)
    x=tf.nn.max_pool3d(conv1_custom_bn_relu,[1,2,3,3,1],strides=[1,2,2,2,1],padding='SAME')
    b1=make_block(x,64,3,64,cnt)
    x=b1.infer()
    cnt=b1.cnt
   
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    b2=make_block(x,128,8,256,cnt,stride=2)
    x=b2.infer()
    cnt=b2.cnt
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    b3=make_block(x,256,36,512,cnt,stride=2)
    x=b3.infer()
    cnt=b3.cnt
    x=tf.nn.max_pool3d(x,[1,2,1,1,1],strides=[1,2,1,1,1],padding='SAME')
    
    shape=x.shape.as_list()
    x=tf.reshape(x,shape=[-1,shape[2],shape[3],shape[4]])
    
    x=make_block(x,512,3,1024,cnt,stride=2).infer()
    
    #Caution:make sure avgpool on the input which has the same shape as kernelsize has been setted padding='VALID'
    x=tf.nn.avg_pool(x,[1,5,5,1],strides=[1,1,1,1],padding='VALID')
    
    x=tf.reshape(x,shape=[-1,2048])
    if(IS_TRAIN):
        x=tf.nn.dropout(x,keep_prob=1)
    else:
        x=tf.nn.dropout(x,keep_prob=1)
    
    x=tf.layers.dense(x,NUM_CLASS)
    
    return x

In [8]:
def save_model_to_serving(inputs, outputs, sess, saver, export_version, export_path): 
    inputs = tf.saved_model.utils.build_tensor_info(inputs)
    outputs = tf.saved_model.utils.build_tensor_info(outputs)
     
    signature = tf.saved_model.signature_def_utils.build_signature_def( 
        inputs={'x': inputs}, outputs={'logit': outputs}, 
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME) 
    print tf.saved_model.signature_constants.PREDICT_METHOD_NAME
    export_path = export_path+'/'+export_version
    builder = tf.saved_model.builder.SavedModelBuilder(export_path) 
    main_op = tf.group(tf.tables_initializer(), name='main_op')
    builder.add_meta_graph_and_variables(sess=sess, 
                                         tags=[tf.saved_model.tag_constants.SERVING], 
                                         signature_def_map={ 'predict_image': signature}, 
                                         main_op=main_op,
                                         saver=saver) 
    builder.save()



In [9]:
def pretreat(param):
    mean, variance = tf.nn.moments(param, axes=[0,1,2])
    std = tf.sqrt(variance) 
    std = tf.maximum(std, tf.constant(1.0/np.sqrt(160*160*3), dtype=tf.float32))
    param = tf.subtract(param, mean)
    param = tf.divide(param, std)
    return param
    
def decode(param):
    return tf.image.decode_jpeg(param, channels=3, dct_method='INTEGER_ACCURATE')
    
def get_inputs(param):
    
    my_input = tf.map_fn(elems=param, fn=decode, dtype=tf.uint8)

    my_input = tf.reshape(my_input,[16, 160, 160, 3])
    my_input = tf.cast(my_input, dtype=tf.float32)
    
    my_input = tf.map_fn(elems=my_input, fn=pretreat, dtype=tf.float32)
    return my_input

MOVING_AVERAGE_DECAY=0.99
tf.reset_default_graph()
IS_TRAIN=False

with tf.Graph().as_default():
    
    global_step=tf.get_variable('global_step',[],initializer=tf.constant_initializer(0),trainable=False)
    root_input_placeholder =tf.placeholder(tf.string,shape=(None,16))
    input_placeholder = tf.map_fn(elems=root_input_placeholder, fn=get_inputs, dtype=tf.float32)
    input_placeholder = tf.reshape(input_placeholder,[-1,16,160,160,3])
    logit=inference_p3d(input_placeholder)
    variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,num_updates=global_step)

    init=tf.global_variables_initializer()
    variable_avg_restore=variable_averages.variables_to_restore()
    
    saver=tf.train.Saver(variable_avg_restore)
    
    config = tf.ConfigProto()  
    config.gpu_options.allow_growth=True
    sess=tf.Session(config=config)
    sess.run(init)
    #restore your checkpoint file
    saver.restore(sess,'./SC_DA_P3D_RES1010_30000-30000')

    save_model_to_serving(root_input_placeholder, logit, sess, saver, '1111', './3daction')

INFO:tensorflow:Restoring parameters from ./SC_DA_P3D_RES1010_30000-30000
tensorflow/serving/predict
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./3daction/1111/saved_model.pb
